# 1. Captura do console do javascript para obter resultados do glpk no wasm

In [2]:
import js
# interceptar console
js_code = r'''
// Variável para armazenar as saídas capturadas
var console_output = "";

// Função para interceptar console.log
function extendLog(logger) {
    var original = console.log;
    console.log = function() {
        logger(Array.prototype.slice.call(arguments));
        return original.apply(this, arguments);
    }
}
function recorder(args) {
    console_output += args.join(' ') + '\n'
}
function cls() {
    console_output = '';
}
extendLog(recorder);
'''
js.eval(js_code)

# 2. Importação do glpk wasm

In [3]:
with open('js/glpsol.js', 'r') as f:
    js_code = f.read()
js.eval(js_code)

# 3. Formulação do problema a) no formato cplex para o solver glpk

In [13]:
from js import FS, callMain, console_output
PPL = '''Minimize
  d1 + d2 + d3 + d4 + d5 + d6 + d7 + d8
Subject To
   c1: d1 - X1 >= -3
   c2: d1 + X1 >= 3
   c3: d2 - X2 >= -1
   c4: d2 + X2 >= 1
   c5: d3 - X1 >= 0
   c6: d3 + X1 >= 0
   c7: d4 - X2 >= 3
   c8: d4 + X2 >= -3
   c9: d5 - X1 >= 2
  c10: d5 + X1 >= -2
  c11: d6 - X2 >= -2
  c12: d6 + X2 >= 2
  c13: d7 - X1 >= -1
  c14: d7 + X1 >= 1
  c15: d8 - X2 >= -4
  c16: d8 + X2 >= 4
  c17: -4 X1 + 3 X2 >= -9
  c18: 5 X1  + 2 X2 >= -6
  c19: 2 X1  - 3 X2 >= -10
  c20: -3 X1 - 2 X2 >= -11
Bounds
  X1 free
  X2 free
  d1 >= 0
  d2 >= 0
  d3 >= 0
  d4 >= 0
  d5 >= 0
  d6 >= 0
  d7 >= 0
  d8 >= 0
End'''

FS.writeFile('/problem.lp', PPL)

# Resolucao pelo GLPK:

In [14]:
js.eval('''callMain(["--lp", "/problem.lp", "-o", "/solution"])''')

0

# Lendo a solução

In [15]:
print(
    FS.readFile('/solution', encoding='utf8')
)

Problem:    
Rows:       20
Columns:    10
Non-zeros:  40
Status:     OPTIMAL
Objective:  obj = 14 (MINimum)

   No.   Row name   St   Activity     Lower bound   Upper bound    Marginal
------ ------------ -- ------------- ------------- ------------- -------------
     1 c1           B              3            -3               
     2 c2           NL             3             3                           1 
     3 c3           NL            -1            -1                           1 
     4 c4           B              3             1               
     5 c5           NL             0             0                           1 
     6 c6           B              0             0               
     7 c7           NL             3             3                           1 
     8 c8           B              7            -3               
     9 c9           NL             2             2                           1 
    10 c10          B              2            -2               
    1